In [1]:
import yfinance as yf
import pandas as pd

/Users/dabereabasse/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [20]:
# Download Apple stock data at 1 hour intervals
start_date = "2023-01-01"
end_date = "2024-11-30"

data = yf.download(tickers='AAPL', interval='1h', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [21]:
# Convert index to column
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Datetime'])

# Keep only the closing price
data = data[['Date', 'Close']]
data.head()

,Date,Close
0,2023-01-03 09:30:00-05:00,125.459999
1,2023-01-03 10:30:00-05:00,125.345001
2,2023-01-03 11:30:00-05:00,124.660004
3,2023-01-03 12:30:00-05:00,124.649902
4,2023-01-03 13:30:00-05:00,124.570000


In [ ]:
# Calculate indicators
def rate_of_change(data, period):
    return data['Close'].pct_change(periods=period) * 100

def relative_strength_index(data, period):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

def rolling_standard_deviation(data, period):
    return data['Close'].rolling(window=period).std()

In [24]:
data['ROC_1'] = rate_of_change(data, 1)
data['ROC_2'] = rate_of_change(data, 2)
data['RSI_14'] = relative_strength_index(data, 14)
data['RSI_28'] = relative_strength_index(data, 28)
data['STD_14'] = rolling_standard_deviation(data, 14)
data['STD_28'] = rolling_standard_deviation(data, 28)

data = data.dropna()

In [25]:
data.head()

,Date,Close,ROC_1,ROC_2,RSI_14,RSI_28,STD_14,STD_28
54,2023-01-12 14:30:00-05:00,133.184998,-0.009382,0.282358,66.767942,60.061781,0.804610,1.517032
55,2023-01-12 15:30:00-05:00,133.389999,0.153923,0.144526,65.251272,60.506327,0.602820,1.486625
56,2023-01-13 09:30:00-05:00,133.509995,0.089958,0.244019,59.926644,53.049848,0.573241,1.514103
57,2023-01-13 10:30:00-05:00,133.029999,-0.359520,-0.269886,58.790865,49.822009,0.438099,1.512512
58,2023-01-13 11:30:00-05:00,133.510101,0.360898,0.000080,55.061219,52.453438,0.445965,1.535557
